## Imports

In [ ]:
import ml_pdf.py
import gen_pdfs.py
import dns_plotter.py
import utilities

## Plot DNS data

In [ ]:
fdir = '/projects/exact/Shashank/plt_DRM_0.7_1095_ML_Output'
plot_dns(fdir)

## Generate the PDFs from the DNS data

In [ ]:
dice = "dice_0004"
datadir = os.path.abspath('data')
pdf, bins, means = gen_pdf_from_dice(os.path.join(datadir, f"{dice}.npz")) 

Alternatively, load the pdf, bins, and means (if they have already been generated)

In [ ]:
pdf = pd.read_pickle(os.path.join(datadir, f"{dice}_pdfs.gz"))
bins = pd.read_pickle(os.path.join(datadir, "bins.gz"))
means = pd.read_pickle(os.path.join(datadir, f"{dice}_src_pv_means.gz"))

If you have all the dice, you can concatenate them into one large dataframe

In [ ]:
dices = ["dice_0001","dice_0002","dice_0003","dice_0004","dice_0005"]
pdf = pd.concat([pd.read_pickle(os.path.join(datadir, f"{dice}_pdfs.gz")) for dice in dices], ignore_index=True)
means = pd.concat([pd.read_pickle(os.path.join(datadir, f"{dice}_src_pv_means.gz")) for dice in dices], ignore_index=True)
pdf.to_pickle(os.path.join(datadir, "dices_pdfs.gz"))
means.to_pickle(os.path.join(datadir, "dices_src_pv_means.gz"))

This is how to get the bin edges

In [ ]:
cbin_edges = utilities.midpoint_to_edges(np.unique(bins.Cbins))
zbin_edges = utilities.midpoint_to_edges(np.unique(bins.Zbins))

Plot slices in the dices, the input space and some sample pdfs

In [ ]:
[plot_dice_slices(os.path.join(datadir, f"{dice}.npz")) for dice in dices]

for dice in dices:
    pdf = pd.read_pickle(os.path.join(datadir, f"{dice}_pdfs.gz"))
    plot_input_space(pdf, fname=f"inputs_{dice}.pdf")
    
# Find PDFs with points closest to these:
points = pd.DataFrame({'Z':[0, 0.4, 0.6255, 0.6714,0.8, 0.9252],
                       'Zvar': [0, 0.0066, 0.0134, 0.0128, 0.01, 0.0043],
                       'C':[0, 0.0269, 0.0318, 0.0822, 0.05, 0.1209],
                       'Cvar':[0, 0.0006, 0.0016, 0.0034, 0.0029, 0.0046]})
idx = [closest_point(points.loc[i,:], pdf.loc[:,points.columns]).name for i in points.index]
plot_pdfs(pdf.loc[idx], means.loc[idx], bins)

# Or (fewer points)
points = pd.DataFrame({'Z':[0, 0.4, 0.6714, 0.9252],
                       'Zvar': [0, 0.0066, 0.0128, 0.0043],
                       'C':[0, 0.0269, 0.0822, 0.1209],
                       'Cvar':[0, 0.0006, 0.0034, 0.0046]})
idx = [closest_point(points.loc[i,:], pdf.loc[:,points.columns]).name for i in points.index]
plot_pdfs(pdf.loc[idx], means.loc[idx], bins)

Find distances between PDFs in different dice

In [ ]:
distances = pdf_distances("dice_0004")
plot_pdf_distances("dice_0004")

## Generate the training data

In [ ]:
Xtrain, Xdev, Xtest, Ytrain, Ydev, Ytest, scaler = gen_training(pdf, dice)

Alternatively, load the training data (if it has already been generated)

In [ ]:
Xtrain = pd.read_pickle(os.path.join(datadir, f"{dice}_xtrain.gz"))
Xdev = pd.read_pickle(os.path.join(datadir, f"{dice}_xdev.gz"))
Ytrain = pd.read_pickle(os.path.join(datadir, f"{dice}_ytrain.gz"))
Ydev = pd.read_pickle(os.path.join(datadir, f"{dice}_ydev.gz"))

Sometimes, one might need to switch scalers (e.g. you train on one dice and want to predict on another)

In [ ]:
scaler_dice_0002 = joblib.load(os.path.join(datadir, "dice_0002_scaler.pkl"))
scaler_dice_0003 = joblib.load(os.path.join(datadir, "dice_0003_scaler.pkl"))
Xtrain = utilities.switch_scaler(Xtrain, scaler_dice_0003, scaler_dice_0002)
Xdev = utilities.switch_scaler(Xdev, scaler_dice_0003, scaler_dice_0002);

## PDF predictions with machine learning

### Random Forest

In [ ]:
mtrain_rf, mdev_rf, RF = rf_training(Xtrain, Xdev, Ytrain, Ydev)
plot_result( Ytrain, mtrain_rf, Ydev, mdev_rf, pdf.loc[Xdev.index,Xdev.columns], bins, fname = "RF.pdf")
conv_rf = convolution_means(mdev_rf, means.loc[Ydev.index])
plot_scatter(pdf.SRC_PV.loc[Ydev.index], conv_rf, fname = "convolution_RF.pdf")

### Linear regression

In [ ]:
mtrain_lr, mdev_lr, LR = lr_training(Xtrain, Xdev, Ytrain, Ydev)

### Polynomial regression

In [ ]:
mtrain_pr, mdev_pr, PR = pr_training(Xtrain, Xdev, Ytrain, Ydev, order=6)

### Feed-forward Neural Network

In [ ]:
mtrain_dnn, mdev_dnn, DNN = dnn_training(Xtrain, Xdev, Ytrain, Ydev, use_gpu=True)

Alternatively, load a pretrained network

In [ ]:
device = torch.device("cpu")
dtype = torch.double
vh = VariableHandler(device=device, dtype=dtype)
batch_size = 64
input_size = Xtrain.shape[1]
layer_sizes = [256, 512, Ytrain.shape[1]]
DNN = Net(input_size, layer_sizes, vh).to(device=device, dtype=dtype)
DNN.load('DNN.pkl')
mtrain_dnn = DNN.predict(Xtrain)
mdev_dnn = DNN.predict(Xdev)

In [ ]:
plot_result( Ytrain, mtrain_dnn, Ydev, mdev_dnn, pdf.loc[Xdev.index,Xdev.columns], bins, fname = "DNN.pdf")
conv_dnn = convolution_means(mdev_dnn, means.loc[Ydev.index])
plot_scatter(pdf.SRC_PV.loc[Ydev.index], conv_dnn, fname = "convolution_DNN.pdf")

Estimate of feature importance through the shuffled input loss

In [ ]:
imp_dnn = shuffled_input_loss(DNN, Xdev, Ydev)
imp_dnn.div(imp_dnn.original, axis=0)

## PDF predictions with generative models

### Conditional Variational Autoencoder

In [ ]:
mtrain_cvae, mdev_cvae, cvae = cvae_training(Xtrain, Xdev, Ytrain, Ydev, use_gpu=True)

Alternatively, load a pre-trained model:

In [ ]:
device = torch.device("cpu")
vh = VariableHandler(device=device, dtype=torch.double)
nlabels = Xtrain.shape[1]
input_size = Ytrain.shape[1]
batch_size = 64
encoder_layer_sizes = [input_size + nlabels, 512, 256]
latent_size = 10
decoder_layer_sizes = [256, 512, input_size]

cvae = CVAE(
        encoder_layer_sizes=encoder_layer_sizes,
        latent_size=latent_size,
        decoder_layer_sizes=decoder_layer_sizes,
        nlabels=nlabels,
        vh=vh,
    ).to(device=device)
cvae.load("CVAE.pkl")

mtrain_cvae = cvae.predict(Xtrain)
mdev_cvae = cvae.predict(Xdev)

In [ ]:
plot_result( Ytrain, mtrain_cvae, Ydev, mdev_cvae, pdf.loc[Xdev.index,Xdev.columns], bins, fname='CVAE.pdf')
conv_cvae = convolution_means(mdev_cvae, means.loc[Ydev.index])
plot_scatter(pdf.SRC_PV.loc[Ydev.index], conv_cvae, fname = "convolution_CVAE.pdf")

You can also use the model to predict on all the dices

In [ ]:
scaler_dice_0002 = joblib.load(os.path.join(datadir, "dice_0002_scaler.pkl"))
predict_all_dice(cvae, scaler_dice_0002)

### Conditional Generative Adversarial Network

In [ ]:
mtrain_cgan, mdev_cgan, cgan = cgan_training(Xtrain, Xdev, Ytrain, Ydev, use_gpu=True)
plot_result( Ytrain, mtrain_cgan, Ydev, mdev_cgan, pdf.loc[Xdev.index,Xdev.columns], bins, fname='CGAN.pdf')
conv_cgan = convolution_means(mdev_cgan, means.loc[Ydev.index])
plot_scatter(pdf.SRC_PV.loc[Ydev.index], conv_cgan, fname = "convolution_CGAN.pdf")

## PDF predictions with analytical models

### delta-delta model

In [ ]:
dd = DD(zbin_edges, cbin_edges)
mtrain_dd = dd.predict(pdf.loc[Xtrain.index,['C','Z']])
mdev_dd = dd.predict(pdf.loc[Xdev.index,['C','Z']])
summarize_training(Ytrain, mtrain_dd, Ydev, mdev_dd, fname="DD.log")
plot_result( Ytrain, mtrain_dd, Ydev, mdev_dd, pdf.loc[Xdev.index,Xdev.columns], bins, fname = "DD.pdf")
conv_dd = convolution_means(mdev_dd, means.loc[Ydev.index])
plot_scatter(pdf.SRC_PV.loc[Ydev.index], conv_dd, fname = "convolution_DD.pdf")

### beta-delta model

In [ ]:
bd = BD(zbin_edges, cbin_edges)
mtrain_bd = bd.predict(pdf.loc[Xtrain.index,['C','Z','Zvar']])
mdev_bd = bd.predict(pdf.loc[Xdev.index,['C','Z','Zvar']])
summarize_training(Ytrain, mtrain_bd, Ydev, mdev_bd, fname="BD.log")
plot_result( Ytrain, mtrain_bd, Ydev, mdev_bd, pdf.loc[Xdev.index,Xdev.columns], bins, fname = "BD.pdf")
conv_bd = convolution_means(mdev_bd, means.loc[Ydev.index])
plot_scatter(pdf.SRC_PV.loc[Ydev.index], conv_bd, fname = "convolution_BD.pdf")

### beta-beta model

In [ ]:
bb = BB(zbin_edges, cbin_edges)
mtrain_bb = bb.predict(pdf.loc[Xtrain.index,['C','Cvar','Z','Zvar']])
mdev_bb = bb.predict(pdf.loc[Xdev.index,['C','Cvar','Z','Zvar']])
summarize_training(Ytrain, mtrain_bb, Ydev, mdev_bb, fname="BB.log")
plot_result( Ytrain, mtrain_bb, Ydev, mdev_bb, pdf.loc[Xdev.index,Xdev.columns], bins, fname = "BB.pdf")
conv_bb = convolution_means(mdev_bb, means.loc[Ydev.index])
plot_scatter(pdf.SRC_PV.loc[Ydev.index], conv_bb, fname = "convolution_BB.pdf")

Good, medium, bad beta models:

In [ ]:
# Find index
m_bb = bb.predict(pdf.loc[:,['C','Cvar','Z','Zvar']])
jsd_bb = calculate_jsd(pdf.loc[:,Ytrain.columns], m_bb)
idx = [jsd_bb.argmin(), np.fabs(jsd_bb - np.log(2)/2).argmin(), jsd_bb.argmax()]

# Plot PDFs
for i, index in enumerate(idx):
    m_bb = {'BB': bb.predict(pdf.loc[[index],['C','Cvar','Z','Zvar']])}
    plot_pdfs(pdf.loc[[index]], means.loc[[index]], bins, fname=f"pdfs_{index}.pdf", models=m_bb)

## Training and predicting on a subset of the data

In [ ]:
idx = pdf.xc < 0
Xtrain_sub = Xtrain.loc[idx.loc[Xtrain.index]]
Xdev_sub = Xdev.loc[idx.loc[Xdev.index]]
Ytrain_sub = Ytrain.loc[idx.loc[Ytrain.index]]
Ydev_sub = Ydev.loc[idx.loc[Ydev.index]]

mtrain_dnn, mdev_dnn, DNN = dnn_training(Xtrain_sub, Xdev_sub, Ytrain_sub, Ydev_sub, use_gpu=True)

dnn_h = predict_all_dice(DNN, scaler_dice_0004, half=True)
plot_dice_predictions({'DNN':dnn_h})

## Prediction timings

In [ ]:
# Load all the models and then:
pt = prediction_times({'RF':RF, 'DNN':DNN, 'CVAE': cvae}, Xdev, Ydev)
pt.loc[:,['model','time','error']].to_latex()

# For the analytical models, you can do
pt = prediction_times({'BB': bb}, pdf.loc[Xdev.index,['C','Cvar','Z','Zvar']], Ydev)

## Summary graphs

JSD plots

In [ ]:
jsd = pd.DataFrame({'RF': calculate_jsd(Ydev, mdev_rf), 
                    'DNN': calculate_jsd(Ydev, mdev_dnn), 
                    'CVAE': calculate_jsd(Ydev, mdev_cvae), 
                    'BB': calculate_jsd(Ydev, mdev_bb)})
plot_jsd(jsd)

Convolution plots

In [ ]:
convolutions = pd.DataFrame({'RF': convolution_means(mdev_rf, means.loc[Ydev.index]), 
                             'DNN': convolution_means(mdev_dnn, means.loc[Ydev.index]), 
                             'CVAE': convolution_means(mdev_cvae, means.loc[Ydev.index]),
                             'BB': convolution_means(mdev_bb, means.loc[Ydev.index])})
plot_convolution(pdf.loc[Ydev.index], convolutions, bins)

Good, bad, medium PDFs 

In [ ]:
# based on BB predictions (use with dice_0004)
jsd_bb = calculate_jsd(Ydev, mdev_bb)
idx = [jsd_bb.argmin(), np.fabs(jsd_bb - np.log(2)/2).argmin(), jsd_bb.argmax()]
for i, index in zip(idx, Ydev.index[idx]):
    model_pdfs = {'RF': mdev_rf[np.newaxis, i,:],
                  'DNN': mdev_dnn[np.newaxis, i,:],
                  'CVAE': mdev_cvae[np.newaxis, i,:],
                  'BB': mdev_bb[np.newaxis, i,:]}
    plot_pdfs(pdf.loc[[index]], means.loc[[index]], bins, fname=f"pdfs_{index}.pdf", models=model_pdfs)
    
# based on PDF of DNN predictions and higher filtered reaction rates (use with dices_skip)
omega_lim = 15
jsd_dnn = calculate_jsd(Ydev, mdev_dnn)

points = [jsd_dnn[pdf.SRC_PV.loc[Ydev.index].values > omega_lim].min(),
          np.median(jsd_dnn[pdf.SRC_PV.loc[Ydev.index].values > omega_lim]),
          jsd_dnn[pdf.SRC_PV.loc[Ydev.index].values > omega_lim][np.fabs(jsd_dnn[pdf.SRC_PV.loc[Ydev.index].values > omega_lim] - 0.1).argmin()],
          jsd_dnn[pdf.SRC_PV.loc[Ydev.index].values > omega_lim].max()]
idx = [np.fabs(jsd_dnn - point).argmin() for point in points]

src_pv_err_dnn = np.fabs(pdf.SRC_PV.loc[Ydev.index] - convolution_means(mdev_dnn, means.loc[Ydev.index])).values


for i, index in zip(idx, Ydev.index[idx]):
    model_pdfs = {'RF': mdev_rf[np.newaxis, i,:],
                  'DNN': mdev_dnn[np.newaxis, i,:],
                  'CVAE': mdev_cvae[np.newaxis, i,:],
                  'BB': mdev_bb[np.newaxis, i,:]}
    plot_pdfs(pdf.loc[[index]], means.loc[[index]], bins, fname=f"pdfs_{index}.pdf", models=model_pdfs)

Predictions across dices (load models first)

In [ ]:
bbp = predict_all_dice(bb, None)
rf_4 = predict_all_dice(RF, scaler_dice_0004)
dnn_4 = predict_all_dice(DNN, scaler_dice_0004)
cvae_4 = predict_all_dice(cvae, scaler_dice_0004)
predictions_4 = {'RF': rf_4, 'DNN':dnn_4, 'CVAE': cvae_4, 'BB': bbp}
with open(os.path.join(datadir, 'predictions_4.pkl'), 'wb') as f:
    pickle.dump(predictions_4, f, pickle.HIGHEST_PROTOCOL)
# or load
with open(os.path.join(datadir, 'predictions_4.pkl'), 'rb') as f:
    predictions_4 = pickle.load(f)
# plot
plot_dice_predictions(predictions_4)

Layerwise relevance propagation (LRP)

In [ ]:
scaler_dices_skip = joblib.load(os.path.join(datadir, "dices_skip_scaler.pkl"))
lrps = lrp_all_dice(DNN, scaler_dices_skip)